#preprocessing data

In [2]:
spark.catalog.clearCache()
df = sqlContext.read.format('csv').option("header", 'true').load("/FileStore/tables/allCom.csv").drop("PatientID").withColumnRenamed("duration_next_week","label").drop("PatientID").drop("register_NextWeekNR").drop("assessment_date")
df.cache()

# df = df.filter(col('duration_next_week') < 1000)

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.sql.functions import col  # for indicating a column using a string in the line below
from pyspark.ml.evaluation import RegressionEvaluator

df = df.select([col(c).cast("double").alias(c) for c in df.columns])
# test = test.select([col(c).cast("double").alias(c) for c in test.columns])
df.printSchema()

train, test = df.randomSplit([0.7, 0.3])

from pyspark.ml.feature import VectorAssembler, VectorIndexer
featuresCols = df.columns
featuresCols.remove('label')

vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=2)

root
-- Estimated_CareMoments_Weekly: double (nullable = true)
-- Estimated_Minutes_Weekly: double (nullable = true)
-- Case_management: double (nullable = true)
-- psychosocial_num: double (nullable = true)
-- physiological_num: double (nullable = true)
-- healthRelated_num: double (nullable = true)
-- ass_year_day: double (nullable = true)
-- label: double (nullable = true)
-- duration_mean_each_time: double (nullable = true)
-- duration_median_each_time: double (nullable = true)
-- care_times_weekly: double (nullable = true)
-- duration_week_minus: double (nullable = true)
-- duration_each_time: double (nullable = true)
-- Age: double (nullable = true)
-- estimated_care_duration_3_to_6_months: double (nullable = true)
-- estimated_care_duration_Less_than_3_months: double (nullable = true)
-- estimated_care_duration_Longer_than_6_months: double (nullable = true)
-- estimated_care_duration_Unknown: double (nullable = true)
-- estimated_care_request_Decreasing_Care: double (nullable = true)
-- estimated_care_request_Increasing_Care: double (nullable = true)
-- estimated_care_request_Stable_care_demand: double (nullable = true)
-- FinancerPGB: double (nullable = true)
-- FinancerWLZ: double (nullable = true)
-- FinancerZVW: double (nullable = true)
-- TeamID_0: double (nullable = true)
-- TeamID_219: double (nullable = true)
-- TeamID_222: double (nullable = true)
-- TeamID_313: double (nullable = true)
-- TeamID_346: double (nullable = true)
-- TeamID_426: double (nullable = true)
-- TeamID_461: double (nullable = true)
-- TeamID_483: double (nullable = true)
-- TeamID_485: double (nullable = true)
-- TeamID_508: double (nullable = true)
-- TeamID_540: double (nullable = true)
-- TeamID_589: double (nullable = true)
-- TeamID_603: double (nullable = true)
-- TeamID_607: double (nullable = true)
-- TeamID_608: double (nullable = true)
-- TeamID_635: double (nullable = true)
-- TeamID_640: double (nullable = true)
-- TeamID_647: double (nullable = true)
-- TeamID_745: double (nullable = true)
-- TeamID_749: double (nullable = true)
-- TeamID_852: double (nullable = true)
-- TeamID_854: double (nullable = true)
-- TeamID_857: double (nullable = true)
-- TeamID_886: double (nullable = true)
-- TeamID_970: double (nullable = true)
-- TeamID_1010: double (nullable = true)
-- TeamID_1040: double (nullable = true)
-- ProblemNameCognition: double (nullable = true)
-- ProblemNameConsciousness: double (nullable = true)
-- ProblemNameDigestion_and_hydratation: double (nullable = true)
-- ProblemNameInterpersonal_relationship: double (nullable = true)
-- ProblemNameMedication_regimen: double (nullable = true)
-- ProblemNameMental_health: double (nullable = true)
-- ProblemNameNutrition: double (nullable = true)
-- ProblemNamePersonal_care: double (nullable = true)
-- ProblemNameReproductive_function: double (nullable = true)
-- ProblemNameRespiration: double (nullable = true)
-- ProblemNameRole_change: double (nullable = true)
-- ProblemNameSkin: double (nullable = true)
-- ProblemNameSpeech_and_language: double (nullable = true)
-- ProblemNameVision: double (nullable = true)
-- combindSymptonProblemDigestion_and_hydratation_Other: double (nullable = true)
-- combindSymptonProblemMedication_regimen_Other: double (nullable = true)
-- combindSymptonProblemMedication_regimen_unable_to_take_medications_without_help: double (nullable = true)
-- combindSymptonProblemomittedSymptons: double (nullable = true)
-- combindSymptonProblemPain_expresses_discomfort_pain: double (nullable = true)
-- combindSymptonProblemPersonal_care_difficulty_with_bathing: double (nullable = true)
-- combindSymptonProblemSkin_drainage: double (nullable = true)
-- combindSymptonProblemSkin_lesion_pressure_ulcer: double (nullable = true)
-- combindSymptonProblemSkin_others: double (nullable = true)

#linear regression

In [4]:
## linear regression
from pyspark.ml.regression import LinearRegression
# Load training data
lr = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, lr])
pipelineModel = pipeline.fit(train)

# Print the coefficients and intercept for linear regression
lrModel = pipelineModel.stages[-1]
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Coefficients: [-2.72997373335,0.191224413392,6.91432260457,9.1074792516,0.789762877782,-21.926222515,-0.0683526284018,7.80691712865,-2.50418077744,5.08839180648,0.564719289425,-0.569436207068,2.37820952433,14.3644818309,-9.78448738141,-48.7382208335,-0.310264193682,-18.7122989153,99.2746660307,56.2027244442,-107.82697547,-73.6460756254,-26.7504493756,20.1610217858,-187.386397829,14.0079451167,33.005276273,86.0643329973,104.658113416,44.6203790814,-9.30190820951,40.2242493023,-14.5549812653,70.889976411,-0.0,44.7422153622,-49.6420030363,8.31944544714,7.34293290764,-31.2775841385,96.614424934,-477.117814597,591.875502788,88.3489294306,14.3999040631,0.0,27.2837296167,86.9943141353,-42.9521351997,159.038898218,98.987043774,76.1662316834,-2.45057224976,-18.6696270598,-12.4907224238,-9.3069505673,30.6404388155,15.6544900023,76.1109131637,-5.4198051729,6.10789866904,7.50815736277,26.3615846132,58.8441268344,17.182919209,-16.7830074056,6.96195510627,-9.22019963552,-16.7169599164,-11.752876825,-36.2099210365,-32.3737367496,-37.7516586983]
Intercept: -286.220648624
numIterations: 11
objectiveHistory: [0.49999999999999994, 0.42294952477623265, 0.16141773860144137, 0.1553129015141799, 0.13926387711204963, 0.13798283965810473, 0.13608543783803909, 0.13537748209893177, 0.13475549119867897, 0.13363352257729494, 0.13317988481838477]
+-------------------+
 residuals|
+-------------------+
 65.80108260768435|
 65.80108260768435|
 -47.90209361521636|
 -67.2389056845434|
 -45.05491257987501|
 -64.39172464920205|
 -64.39172464920205|
 -42.20773154453366|
 -42.20773154453366|
 -61.5445436138607|
 -9.669001035972087|
-22.719657167579612|
-17.025295096896798|
-17.025295096896798|
-17.854250194584324|
-17.854250194584324|
-17.854250194584324|
 -48.49468901008248|
-12.325482877072716|
-15.007069159242974|
+-------------------+
only showing top 20 rows

RMSE: 469.190576
r2: 0.734552

In [5]:
predictions = pipelineModel.transform(test)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

##when data below than 1000
# RMSE on our test set: 144.898
# r2 on our test set: 0.611046
# mae on our test set: 102.653

RMSE on our test set: 465.699
r2 on our test set: 0.743918
mae on our test set: 196.332

In [6]:
display(predictions)

Estimated_CareMoments_Weekly,Estimated_Minutes_Weekly,Case_management,psychosocial_num,physiological_num,healthRelated_num,ass_year_day,label,duration_mean_each_time,duration_median_each_time,care_times_weekly,duration_week_minus,duration_each_time,Age,estimated_care_duration_3_to_6_months,estimated_care_duration_Less_than_3_months,estimated_care_duration_Longer_than_6_months,estimated_care_duration_Unknown,estimated_care_request_Decreasing_Care,estimated_care_request_Increasing_Care,estimated_care_request_Stable_care_demand,FinancerPGB,FinancerWLZ,FinancerZVW,TeamID_0,TeamID_219,TeamID_222,TeamID_313,TeamID_346,TeamID_426,TeamID_461,TeamID_483,TeamID_485,TeamID_508,TeamID_540,TeamID_589,TeamID_603,TeamID_607,TeamID_608,TeamID_635,TeamID_640,TeamID_647,TeamID_745,TeamID_749,TeamID_852,TeamID_854,TeamID_857,TeamID_886,TeamID_970,TeamID_1010,TeamID_1040,ProblemNameCognition,ProblemNameConsciousness,ProblemNameDigestion_and_hydratation,ProblemNameInterpersonal_relationship,ProblemNameMedication_regimen,ProblemNameMental_health,ProblemNameNutrition,ProblemNamePersonal_care,ProblemNameReproductive_function,ProblemNameRespiration,ProblemNameRole_change,ProblemNameSkin,ProblemNameSpeech_and_language,ProblemNameVision,combindSymptonProblemDigestion_and_hydratation_Other,combindSymptonProblemMedication_regimen_Other,combindSymptonProblemMedication_regimen_unable_to_take_medications_without_help,combindSymptonProblemomittedSymptons,combindSymptonProblemPain_expresses_discomfort_pain,combindSymptonProblemPersonal_care_difficulty_with_bathing,combindSymptonProblemSkin_drainage,combindSymptonProblemSkin_lesion_pressure_ulcer,combindSymptonProblemSkin_others,rawFeatures,features,prediction
0.0,0.0,0.0,1.0,1.0,1.0,212.0,135.0,38.0,45.0,5.0,190.0,10.0,66.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"List(0, 73, List(3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 22, 48, 57, 69), List(1.0, 1.0, 1.0, 212.0, 38.0, 45.0, 5.0, 190.0, 10.0, 66.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 73, List(3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 22, 48, 57, 69), List(1.0, 1.0, 1.0, 212.0, 38.0, 45.0, 5.0, 190.0, 10.0, 66.0, 1.0, 1.0, 1.0, 1.0, 1.0))",89.1291846397051
0.0,0.0,0.0,1.0,1.0,1.0,212.0,135.0,38.0,45.0,5.0,190.0,45.0,66.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"List(0, 73, List(3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 22, 48, 57, 69), List(1.0, 1.0, 1.0, 212.0, 38.0, 45.0, 5.0, 190.0, 45.0, 66.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 73, List(3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 22, 48, 57, 69), List(1.0, 1.0, 1.0, 212.0, 38.0, 45.0, 5.0, 190.0, 45.0, 66.0, 1.0, 1.0, 1.0, 1.0, 1.0))",69.19891739231565
0.0,0.0,0.0,1.0,1.0,1.0,212.0,135.0,38.0,45.0,5.0,190.0,45.0,66.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,"List(0, 73, List(3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 22, 48, 57, 69), List(1.0, 1.0, 1.0, 212.0, 38.0, 45.0, 5.0, 190.0, 45.0, 66.0, 1.0, 1.0, 1.0, 1.0, 1.0))","List(0, 73, List(3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 16, 22, 48, 57, 69), List(1.0, 1.0, 1.0, 212.0, 38.0, 45.0, 5.0, 190.0, 45.0, 66.0, 1.0, 1.0, 1.0, 1.0, 1.0))",69.19891739231565
1.0,15.0,0.0,1.0,2.0,0.0,66.0,5.0,16.0,15.0,5.0,80.0,15.0,86.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,"List(0, 73, List(0, 1, 3, 4, 6, 7, 8, 9, 10, 11,

#Generalized linear regression

In [8]:
from pyspark.ml.regression import GeneralizedLinearRegression
glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)
#gama: Inverse*, Identity, Log
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, glr])
pipelineModel = pipeline.fit(train)
lrModel = pipelineModel.stages[-1]

print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

summary = lrModel.summary
print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
print("T Values: " + str(summary.tValues))
print("P Values: " + str(summary.pValues))
print("Dispersion: " + str(summary.dispersion))
print("Null Deviance: " + str(summary.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
print("Deviance Residuals: ")
summary.residuals().show()

predictions = pipelineModel.transform(test)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae


Coefficients: [1.85707938539,0.132168129312,13.3053103698,9.32274277445,0.673890667989,-24.1246626895,-0.0400148314319,5.33499550649,-1.94551905759,-2.16801297845,0.725653466534,-0.33279119758,1.95810505821,33.0343664427,2.42464326267,-38.1810170479,15.0628401386,-5.31290217203,124.150908191,66.2941448177,-123.603512026,-79.3141098322,-31.3011881654,2.11091423615,-149.004495749,-27.7134557975,-2.1073997552,76.9434851389,88.4187463451,50.8893872074,-51.2995534634,-3.16051968568,-34.7511227663,45.1091748358,-14.8749596495,16.031398633,-51.1390636599,-9.35754783202,-11.8304594841,-56.9190191873,68.7930931501,-280.543593639,410.61843372,43.4648472209,9.20067837754,-30.4886605697,36.7784017903,71.5400672769,-76.9505040153,146.516332895,113.824679387,86.8697581679,7.37620846794,-21.427571168,-1.66197827517,-5.18548810433,32.1396152445,30.2062403606,53.6803798985,-4.76886883718,10.015772778,17.6012328305,45.498869244,77.7739369674,10.9420978784,-27.4586697102,-8.91954321496,-18.6347442387,-14.0494072459,-27.1583991294,-51.5839241614,-44.2238284684,-51.0808610233]
Intercept: -207.562565129
Coefficient Standard Errors: [0.14014282608567608, 0.0014628781692896946, 1.6099130290865533, 0.7023011024236652, 0.4967370593562454, 0.7210955039619376, 0.0070680088471244415, 0.08530855883202734, 0.04812782060139525, 0.16181408349634782, 0.0028822916255894157, 0.023020759978187626, 0.0689540925575522, 44.75255556299547, 44.74464779475227, 44.73558380985222, 44.73010077954942, 2.5255787571790957, 2.2216515691162835, 2.7339476497320825, 11.798500131583351, 5.431417050480018, 3.735871092829413, 60.81605551821019, 61.01802659735609, 61.43711582559171, 61.353664571878305, 61.594600431433136, 61.50210976816593, 61.347473509758814, 61.53757149713535, 61.54422742000502, 61.24800734262793, 61.441111574175906, 61.44025018074675, 60.95206336461725, 61.482921519661055, 61.50343750688533, 61.10999749617118, 61.46360878200548, 61.21097082849792, 61.48297878717602, 61.88593843074383, 61.463724530803766, 61.46659808186664, 61.518025466293906, 61.229197367521216, 61.437687254461366, 61.526883832936214, 61.35873853558375, 3.7147209183411136, 10.674269120581556, 5.569785192079128, 5.395645242716457, 4.865952182873108, 3.8506651727759706, 3.337204687265579, 3.3475248673452422, 16.632746510986916, 4.604887175950036, 9.756021154900141, 3.625157846819104, 8.69114366985913, 4.328922849210514, 8.900955432240043, 6.884812922595061, 6.371485152370741, 1.9293342154319306, 4.078101147421626, 4.543014372158634, 6.9704167837031035, 5.017793646184061, 7.43000336081769, 75.84062329017566]
T Values: [13.251333923079967, 90.34800852660808, 8.264614379395615, 13.274566624312449, 1.356634572146934, -33.45557219114826, -5.661400869376965, 62.53763490478494, -40.42400078122965, -13.398172344476162, 251.76268080966435, -14.456134284676928, 28.397227569520748, 0.7381559785170191, 0.054188453416688244, -0.8534820336803969, 0.336749523834914, -2.1036374957317787, 55.88225890889423, 24.248505571855986, -10.476205504749691, -14.602839202920089, -8.37855145095252, 0.034709818289995566, -2.441975004072165, -0.45108653661631437, -0.03434839255163418, 1.2491920493021433, 1.437653873637287, 0.8295270252547139, -0.8336298007106595, -0.051353633284151046, -0.5673837284511588, 0.7341855262715482, -0.24210447720718015, 0.26301650425034795, -0.8317604693457792, -0.15214674514689913, -0.19359286481487073, -0.9260604822150013, 1.1238686826725415, -4.562947325793633, 6.635084546379455, 0.7071625996102235, 0.14968582392157456, -0.4956053179308261, 0.6006677103655033, 1.1644329478197915, -1.2506809905117122, 2.387864163955652, 30.641515712478352, 8.13823946038576, 1.324325483579678, -3.9712713130964885, -0.34155252922894286, -1.3466473639390955, 9.630699419531544, 9.02345510716042, 3.227391210646049, -1.035610353731629, 1.0266247498845973, 4.855301085990048, 5.23508423889192, 17.96611759472487, 1.2293172302294977, -3.988295690668015, -1.3999158754447525, -9.658639798986036, -3.445085528282611, -5.978057057

In [9]:
# # Gaussian, identity
# RMSE on our test set: 463.274
# r2 on our test set: 0.746578
# mae on our test set: 195.02
# #possion,log
# RMSE on our test set: 601.665
# r2 on our test set: 0.57787
# mae on our test set: 243.454
# #possion,sqrt
# RMSE on our test set: 531.159
# r2 on our test set: 0.671008
# mae on our test set: 207.285
# #gaussian log/inverse
# RMSE on our test set: 531.159
# r2 on our test set: 0.671008
# mae on our test set: 207.285

#Decision tree regression

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="features")

pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, dt])
pipelineModel = pipeline.fit(train)
# Print the coefficients and intercept for linear regression
treeModel = pipelineModel.stages[-1]

predictions = pipelineModel.transform(test)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

# summary only
print(treeModel)

RMSE on our test set: 392.465
r2 on our test set: 0.818126
mae on our test set: 177.598
DecisionTreeRegressionModel (uid=DecisionTreeRegressor_43139b0db886cb4c1763) of depth 5 with 57 nodes

#Random forest regression

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="features")
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, rf])
pipelineModel = pipeline.fit(train)
# Print the coefficients and intercept for linear regression
treeModel = pipelineModel.stages[-1]

predictions = pipelineModel.transform(test)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

# summary only
print(treeModel)

RMSE on our test set: 384.247
r2 on our test set: 0.825663
mae on our test set: 176.436
RandomForestRegressionModel (uid=RandomForestRegressor_40b790d21b3e13ececbd) with 20 trees

#Gradient-boosted tree regression

In [15]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Train a GBT model.
gbt = GBTRegressor(featuresCol="features", maxIter=10)
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, gbt])
pipelineModel = pipeline.fit(train)
# Print the coefficients and intercept for linear regression
treeModel = pipelineModel.stages[-1]

predictions = pipelineModel.transform(test)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

print(treeModel)

RMSE on our test set: 337.09
r2 on our test set: 0.865313
mae on our test set: 155.324
GBTRegressionModel (uid=GBTRegressor_4fbdac798c1a77d7b435) with 10 trees

In [16]:
predictions = pipelineModel.transform(train)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our train set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our train set: %g" % r2
print "mae on our train set: %g" % mae

print(treeModel)

RMSE on our train set: 320.969
r2 on our train set: 0.876993
mae on our train set: 154.4
GBTRegressionModel (uid=GBTRegressor_433bbd0e58009e3ded5c) with 10 trees

#Survival regression
TBD

In [18]:
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.linalg import Vectors

training = spark.createDataFrame([
    (1.218, 1.0, Vectors.dense(1.560, -0.605)),
    (2.949, 0.0, Vectors.dense(0.346, 2.158)),
    (3.627, 0.0, Vectors.dense(1.380, 0.231)),
    (0.273, 1.0, Vectors.dense(0.520, 1.151)),
    (4.199, 0.0, Vectors.dense(0.795, -0.226))], ["label", "censor", "features"])
quantileProbabilities = [0.3, 0.6]
aft = AFTSurvivalRegression(quantileProbabilities=quantileProbabilities,
                            quantilesCol="quantiles")

model = aft.fit(training)

# Print the coefficients, intercept and scale parameter for AFT survival regression
print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))
print("Scale: " + str(model.scale))
model.transform(training).show(truncate=False)

Coefficients: [-0.496311146665,0.198444376999]
Intercept: 2.6380946151
Scale: 1.54723455744
+-----+------+--------------+------------------+---------------------------------------+
label|censor|features |prediction |quantiles |
+-----+------+--------------+------------------+---------------------------------------+
1.218|1.0 |[1.56,-0.605] |5.718979487634984 |[1.1603238947151617,4.99545601027475] |
2.949|0.0 |[0.346,2.158] |18.076521181495465|[3.6675458454717664,15.789611866277742]|
3.627|0.0 |[1.38,0.231] |7.381861804239096 |[1.4977061305190829,6.44796261233896] |
0.273|1.0 |[0.52,1.151] |13.577612501425325|[2.754762148150694,11.859872224069736] |
4.199|0.0 |[0.795,-0.226]|9.013097744073871 |[1.8286676321297772,7.872826505878406] |
+-----+------+--------------+------------------+---------------------------------------+

#Isotonic regression
not monotonic, try for fun

In [20]:
from pyspark.ml.regression import IsotonicRegression

iso =  IsotonicRegression()
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, iso])
pipelineModel = pipeline.fit(train)
treeModel = pipelineModel.stages[-1]

predictions = pipelineModel.transform(test)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

print("Boundaries in increasing order: %s\n" % str(treeModel.boundaries))
print("Predictions associated with the boundaries: %s\n" % str(treeModel.predictions))

RMSE on our test set: 897.139
r2 on our test set: 0.0496417
mae on our test set: 416.472
Boundaries in increasing order: [0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,14.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,21.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,28.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,35.0,39.0,39.0]

Predictions associated with the boundaries: [15.1453488372,15.1453488372,20.0,25.0,30.0,35.0,40.0,45.0,50.0,55.0,60.0,65.0,70.0,75.0,80.0,85.0,90.0,95.0,100.0,105.0,110.0,115.0,120.0,125.0,130.0,135.0,145.0,150.0,160.0,165.0,175.0,180.0,185.0,190.0,200.0,205.0,210.0,215.0,235.0,245.0,250.0,255.0,260.0,270.0,292.806297786,292.806297786,295.0,300.0,305.0,310.0,315.0,320.0,325.0,330.0,335.0,340.0,345.0,350.0,355.0,360.0,365.0,370.0,375.0,380.0,385.0,390.0,395.0,398.216544749,398.216544749,400.0,405.0,410.0,415.0,420.0,425.0,430.0,435.0,440.0,445.0,450.0,455.0,460.0,465.0,470.0,475.0,480.0,485.0,490.0,495.0,500.0,505.0,510.0,515.0,520.0,525.0,530.0,535.0,540.0,545.0,550.0,555.0,560.0,565.0,570.0,575.0,580.0,585.0,590.0,595.0,600.0,605.0,610.0,615.0,620.0,625.0,630.0,635.0,640.0,645.0,650.0,655.0,660.0,665.0,670.0,675.0,680.0,685.0,690.0,695.0,700.0,705.0,710.0,713.04471462,713.04471462,715.0,720.0,725.0,730.0,735.0,740.0,745.0,750.0,755.0,760.0,765.0,770.0,775.0,780.0,785.0,790.0,800.0,805.0,810.0,820.0,825.0,830.0,835.0,840.0,850.0,855.0,860.0,865.0,870.0,880.0,885.0,890.0,895.0,895.767447131,895.767447131,900.0,905.0,910.0,915.0,920.0,935.0,940.0,945.0,955.0,965.0,970.0,975.0,980.0,985.0,990.0,1000.0,1010.0,1015.0,1030.0,1035.0,1040.0,1045.0,1065.0,1070.0,1075.0,1080.0,1090.0,1105.0,1110.0,1115.0,1120.0,1125.0,1140.0,1145.0,1155.0,1175.0,1200.0,1205.0,1230.0,1235.0,1240.0,1245.0,1260.0,1265.0,1270.0,1285.0,1295.0,1305.0,1310.0,1320.0,1325.0,1330.0,1345.0,1350.0,1355.0,1360.0,1365.0,1370.0,1380.0,1390.0,1394.47206612,1394.47206612,1455.0,1700.0,1710.0,1765.0,1810.0,1860.0,1870.0,1880.0,2038.33586626,2038.33586626,4800.0]

#conclusion:
test multiple simple regression model 
from the above, we can se gbt is the best, achieved 0.87 r2
then use grid search for the regression

In [22]:
from pyspark.ml.regression import GBTRegressor,GBTRegressionModel
gbt = GBTRegressor(labelCol="label")

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator

paramGrid = ParamGridBuilder()\
  .addGrid(gbt.maxDepth, [10,20])\
  .addGrid(gbt.maxIter, [20,30])\
  .build()
evaluator = RegressionEvaluator(metricName="rmse", labelCol=gbt.getLabelCol(), predictionCol=gbt.getPredictionCol())
cv = CrossValidator(estimator=gbt, evaluator=evaluator, estimatorParamMaps=paramGrid,numFolds = 5)

from pyspark.ml import Pipeline,PipelineModel
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])

In [23]:
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, cv])
pipelineModel = pipeline.fit(train)

predictions = pipelineModel.transform(test)
display(predictions.select("label", "prediction", *featuresCols))

rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

# print(pipelineModel.bestModel.getMaxIter())
# maxIter
bestParams = pipelineModel.stages[-1].bestModel.extractParamMap() ##maxDepth: 5 , maxIter:100
print(bestParams)

rfPath = "/tmp/mllib-persistence-example/rfModel"
pipelineModel.save(rfPath)
# pipelineModel = PipelineModel.load(rfPath)

java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:717)
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:111)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:244)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:12)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:188)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:183)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:180)
	at com.databricks.rpc.JettyClient.withAttributionContext(JettyClient.scala:32)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:234)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:164)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:101)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:47)
	at com.databricks.backend.daemon.driver.DriverClient.send(DriverClient.scala:85)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:270)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:89)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:93)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:413)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:37)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:60)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:280)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:12)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:188)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:58)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:183)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:180)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:229)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:280)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

#feature selection:PCA

In [25]:
# 136 features with feature selection 
#25
RMSE on our test set: 346.163
r2 on our test set: 0.859244
mae on our test set: 159.356
  
#40
RMSE on our test set: 342.78
r2 on our test set: 0.861982
mae on our test set: 157.997
  
#50
RMSE on our test set: 343.726
r2 on our test set: 0.861219
mae on our test set: 158.91

#60
RMSE on our test set: 342.148
r2 on our test set: 0.86249
mae on our test set: 158.672
  
#70
RMSE on our test set: 341.94
r2 on our test set: 0.862657
mae on our test set: 158.763

test results:
feature numbers: R2,

3: 0.76 ; 5: 0.79 ; 10：0.83 ; 15：0.83 ; 20：0.84 ; 30：0.85 ; 40: 0.84 ; 35:0.85 ; 37:0.84

result: pca doesnt work =.=
check feature importance, and sub-sampling features

In [27]:
treeModel.featureImportances

Out[ 23 ]: SparseVector(73, {0: 0.1427, 1: 0.0403, 2: 0.0075, 3: 0.0089, 4: 0.029, 5: 0.0575, 6: 0.0966, 7: 0.0746, 8: 0.0966, 9: 0.0935, 10: 0.1687, 11: 0.0, 12: 0.0691, 13: 0.0129, 14: 0.0019, 15: 0.0136, 16: 0.0002, 17: 0.0062, 18: 0.0145, 19: 0.0191, 23: 0.0, 41: 0.001, 42: 0.005, 43: 0.015, 50: 0.0124, 51: 0.0, 54: 0.0, 56: 0.0006, 67: 0.0011, 72: 0.0115})

In [28]:
print(df.columns)

['Estimated_CareMoments_Weekly', 'Estimated_Minutes_Weekly', 'Case_management', 'psychosocial_num', 'physiological_num', 'healthRelated_num', 'ass_year_day', 'label', 'duration_mean_each_time', 'duration_median_each_time', 'care_times_weekly', 'duration_week_minus', 'duration_each_time', 'Age', 'estimated_care_duration_3_to_6_months', 'estimated_care_duration_Less_than_3_months', 'estimated_care_duration_Longer_than_6_months', 'estimated_care_duration_Unknown', 'estimated_care_request_Decreasing_Care', 'estimated_care_request_Increasing_Care', 'estimated_care_request_Stable_care_demand', 'FinancerPGB', 'FinancerWLZ', 'FinancerZVW', 'TeamID_0', 'TeamID_219', 'TeamID_222', 'TeamID_313', 'TeamID_346', 'TeamID_426', 'TeamID_461', 'TeamID_483', 'TeamID_485', 'TeamID_508', 'TeamID_540', 'TeamID_589', 'TeamID_603', 'TeamID_607', 'TeamID_608', 'TeamID_635', 'TeamID_640', 'TeamID_647', 'TeamID_745', 'TeamID_749', 'TeamID_852', 'TeamID_854', 'TeamID_857', 'TeamID_886', 'TeamID_970', 'TeamID_1010', 'TeamID_1040', 'ProblemNameCognition', 'ProblemNameConsciousness', 'ProblemNameDigestion_and_hydratation', 'ProblemNameInterpersonal_relationship', 'ProblemNameMedication_regimen', 'ProblemNameMental_health', 'ProblemNameNutrition', 'ProblemNamePersonal_care', 'ProblemNameReproductive_function', 'ProblemNameRespiration', 'ProblemNameRole_change', 'ProblemNameSkin', 'ProblemNameSpeech_and_language', 'ProblemNameVision', 'combindSymptonProblemDigestion_and_hydratation_Other', 'combindSymptonProblemMedication_regimen_Other', 'combindSymptonProblemMedication_regimen_unable_to_take_medications_without_help', 'combindSymptonProblemomittedSymptons', 'combindSymptonProblemPain_expresses_discomfort_pain', 'combindSymptonProblemPersonal_care_difficulty_with_bathing', 'combindSymptonProblemSkin_drainage', 'combindSymptonProblemSkin_lesion_pressure_ulcer', 'combindSymptonProblemSkin_others']

In [29]:
#11,23,51,54
print(df.columns[11])
print(df.columns[23])
print(df.columns[51])
print(df.columns[54])

duration_week_minus
FinancerZVW
ProblemNameCognition
ProblemNameInterpersonal_relationship

In [30]:
featuresCols = df.columns
featuresCols.remove('label')
featuresCols.remove('duration_week_minus')
featuresCols.remove('FinancerZVW')
featuresCols.remove('ProblemNameCognition')
featuresCols.remove('ProblemNameInterpersonal_relationship')

vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=2)

In [31]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Train a GBT model.
gbt = GBTRegressor(featuresCol="features", maxIter=10)
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, gbt])
pipelineModel = pipeline.fit(train)
# Print the coefficients and intercept for linear regression
treeModel = pipelineModel.stages[-1]

predictions = pipelineModel.transform(test)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

print(treeModel)

RMSE on our test set: 358.378
r2 on our test set: 0.847765
mae on our test set: 164.655
GBTRegressionModel (uid=GBTRegressor_44e292ca299b38b5f9ba) with 10 trees

#feature without selection

In [33]:
spark.catalog.clearCache()
df = sqlContext.read.format('csv').option("header", 'true').load("/FileStore/tables/allCom_without_feature_selection.csv").drop("PatientID").withColumnRenamed("duration_next_week","label").drop("PatientID").drop("register_NextWeekNR").drop("assessment_date")
df = df.fillna({'Marital_statusAlleenstaand':'2'}).fillna({'Marital_statusGehuwd':'2'}).fillna({'Marital_statusSamenwonend':'2'}).fillna({'Marital_statusSamenwonend_met_contract':'2'})
df.cache()

# df = df.filter(col('duration_next_week') < 1000)

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.sql.functions import col  # for indicating a column using a string in the line below
df = df.select([col(c).cast("double").alias(c) for c in df.columns])
# test = test.select([col(c).cast("double").alias(c) for c in test.columns])
df.printSchema()

train, test = df.randomSplit([0.7, 0.3])

from pyspark.ml.feature import VectorAssembler, VectorIndexer
featuresCols = df.columns
featuresCols.remove('label')
featuresCols.remove('Marital_statusAlleenstaand')
featuresCols.remove('Marital_statusGehuwd')
featuresCols.remove('Marital_statusSamenwonend')
featuresCols.remove('Marital_statusSamenwonend_met_contract')

vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="features", maxCategories=3)

root
-- register_WeekNR: double (nullable = true)
-- Estimated_CareMoments_Weekly: double (nullable = true)
-- Estimated_Minutes_Weekly: double (nullable = true)
-- Case_management: double (nullable = true)
-- psychosocial_num: double (nullable = true)
-- physiological_num: double (nullable = true)
-- healthRelated_num: double (nullable = true)
-- ass_year_day: double (nullable = true)
-- label: double (nullable = true)
-- duration_mean_each_time: double (nullable = true)
-- duration_median_each_time: double (nullable = true)
-- care_times_weekly: double (nullable = true)
-- duration_week_minus: double (nullable = true)
-- duration_each_time: double (nullable = true)
-- Age: double (nullable = true)
-- estimated_care_duration_3_to_6_months: double (nullable = true)
-- estimated_care_duration_Less_than_3_months: double (nullable = true)
-- estimated_care_duration_Longer_than_6_months: double (nullable = true)
-- estimated_care_duration_Unknown: double (nullable = true)
-- estimated_care_request_Decreasing_Care: double (nullable = true)
-- estimated_care_request_Increasing_Care: double (nullable = true)
-- estimated_care_request_Stable_care_demand: double (nullable = true)
-- estimated_care_request_Unknown: double (nullable = true)
-- FinancerMSVT: double (nullable = true)
-- FinancerOR: double (nullable = true)
-- FinancerPF: double (nullable = true)
-- FinancerPGB: double (nullable = true)
-- FinancerWLZ: double (nullable = true)
-- FinancerZVW: double (nullable = true)
-- FinancerZVWS1: double (nullable = true)
-- GenderF: double (nullable = true)
-- GenderM: double (nullable = true)
-- Marital_statusAlleenstaand: double (nullable = true)
-- Marital_statusGehuwd: double (nullable = true)
-- Marital_statusSamenwonend: double (nullable = true)
-- Marital_statusSamenwonend_met_contract: double (nullable = true)
-- Living_unitInstitution_with_residence: double (nullable = true)
-- Living_unitOther_multi_person_household: double (nullable = true)
-- Living_unitParental_home: double (nullable = true)
-- Living_unitSingle_person: double (nullable = true)
-- Living_unitSingle_with_children: double (nullable = true)
-- Living_unitUnknown: double (nullable = true)
-- Living_unitWith_partner: double (nullable = true)
-- Living_unitWith_partner_and_children: double (nullable = true)
-- TeamID_0: double (nullable = true)
-- TeamID_219: double (nullable = true)
-- TeamID_222: double (nullable = true)
-- TeamID_313: double (nullable = true)
-- TeamID_330: double (nullable = true)
-- TeamID_346: double (nullable = true)
-- TeamID_426: double (nullable = true)
-- TeamID_461: double (nullable = true)
-- TeamID_483: double (nullable = true)
-- TeamID_485: double (nullable = true)
-- TeamID_508: double (nullable = true)
-- TeamID_540: double (nullable = true)
-- TeamID_589: double (nullable = true)
-- TeamID_603: double (nullable = true)
-- TeamID_607: double (nullable = true)
-- TeamID_608: double (nullable = true)
-- TeamID_635: double (nullable = true)
-- TeamID_640: double (nullable = true)
-- TeamID_647: double (nullable = true)
-- TeamID_745: double (nullable = true)
-- TeamID_749: double (nullable = true)
-- TeamID_852: double (nullable = true)
-- TeamID_854: double (nullable = true)
-- TeamID_857: double (nullable = true)
-- TeamID_886: double (nullable = true)
-- TeamID_970: double (nullable = true)
-- TeamID_1010: double (nullable = true)
-- TeamID_1040: double (nullable = true)
-- ProblemNameAbuse: double (nullable = true)
-- ProblemNameBowel_funtion: double (nullable = true)
-- ProblemNameCaretaking_parenting: double (nullable = true)
-- ProblemNameCirculation: double (nullable = true)
-- ProblemNameCognition: double (nullable = true)
-- ProblemNameCommunicable_infectious_condition: double (nullable = true)
-- ProblemNameCommunication_with_community_resources: double (nullable = true)
-- ProblemNameConsciousness: double (nullable = true)
-- ProblemNameDigestion_and_hydratation: double (nullable = true)
-- ProblemNameGrief: double (nu

In [34]:
# from pyspark.sql.functions import isnan, when, count, col
# train.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

In [35]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Train a GBT model.
gbt = GBTRegressor(featuresCol="features", maxIter=10)
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, gbt])
pipelineModel = pipeline.fit(train)
# Print the coefficients and intercept for linear regression
treeModel = pipelineModel.stages[-1]

predictions = pipelineModel.transform(test)

evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

print(treeModel)

RMSE on our test set: 321.877
r2 on our test set: 0.878301
mae on our test set: 153.39
GBTRegressionModel (uid=GBTRegressor_4c08a92bdbc81e0e2b9a) with 10 trees

use pca

In [37]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import PCA

vectorAssembler = VectorAssembler(inputCols=featuresCols, outputCol="rawFeatures")
vectorIndexer = VectorIndexer(inputCol="rawFeatures", outputCol="featuresBeforePCA", maxCategories=2)

pca = PCA(k=70, inputCol="featuresBeforePCA", outputCol="features")

# Train a GBT model.
gbt = GBTRegressor(featuresCol="features", maxIter=10)
pipeline = Pipeline(stages=[vectorAssembler, vectorIndexer, pca,gbt])
pipelineModel = pipeline.fit(train)
# Print the coefficients and intercept for linear regression
treeModel = pipelineModel.stages[-1]
predictions = pipelineModel.transform(test)
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print "RMSE on our test set: %g" % rmse
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
print "r2 on our test set: %g" % r2
print "mae on our test set: %g" % mae

print(treeModel)

RMSE on our test set: 341.94
r2 on our test set: 0.862657
mae on our test set: 158.763
GBTRegressionModel (uid=GBTRegressor_48529b2138eec0f971cd) with 10 trees